[View in Colaboratory](https://colab.research.google.com/github/opooladz/ece_210_final_project/blob/master/210_final_project_convnet.ipynb)

In [37]:
# Import Libs 
import numpy as np
import matplotlib.pyplot as plt
import scipy.special
import scipy.io as sio
import math
#from  scipy.ndimage import convolve1d

# Reloading any code written in external .py files.
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
# Keras try model 
def try_model(model, data, name, epochs=200, verbose=1):  
  model.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

  model.summary()  
  history = model.fit(data['x_train'], data['y_train'], 
            epochs=epochs, batch_size=64, verbose=verbose, 
            validation_data=(data['x_val'], data['y_val']))  
  score = model.evaluate(data['x_test'], data['y_test'])
  print('Test accuracy: {}'.format(score))
  
#   out_path = 'serialized_models/{}.h5'.format(name)  
#   model.save(out_path)
  
#   with open('serialized_models/{}_history.pickle'.format(name), 'wb') as handle:
#     pickle.dump(history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
  return history

In [0]:
# Comms Stuff Will add more later
def qammod(m,y):
  if(np.all(y>=m)):
    print('qammod: all elements of X must be in range [0,M-1]')
  
  if (~np.all(y == np.fix(y) ) ):
    print("qammod: all elements of X must be integers");
  
  c = np.sqrt(m);
  if( ~(np.all(c == np.fix(c)) and np.all(np.math.log(c, 2) == np.fix(np.math.log(c, 2))) ) ):
    print("qammod: M must be a square of a power of 2");
  
  b = -2*np.fmod(y,c) + c - 1.;
  a = 2*np.floor(y *1./c) - c + 1.;
  x = a + 1.*1j*b
  # lets hold x in a np array([real,imag])
  # x = np.array([a,1.*b]).T
  return [x , y] 

def qammod2(m,size,trials):
  """
  m - highest integer for the randint
  """
  y = np.random.randint(m,size=(trials,size))
  if(np.all(y>=m)):
    print('qammod: all elements of X must be in range [0,M-1]')
  
  if (~np.all(y == np.fix(y) ) ):
    print("qammod: all elements of X must be integers");
  
  c = np.sqrt(m);
  if( ~(np.all(c == np.fix(c)) and np.all(np.math.log(c, 2) == np.fix(np.math.log(c, 2))) ) ):
    print("qammod: M must be a square of a power of 2");
  
  b = -2*np.fmod(y,c) + c - 1.;
  a = 2*np.floor(y *1./c) - c + 1.;
  #x = a + 1.*1j*b
  # lets hold x in a dp array([real,imag])
  #x = np.array([a,1.*b])
  x = np.dstack((a,b))
  return [x , y]                        

def channel2(m,sequenceLen,trials,Noise,SNRdB):
  
  [x,y] = qammod2(m,sequenceLen,trials)
  h = [1, 0.5, 0.3, -0.5, 0.2, 0, 0.4]
  #h = [1]
  #print(x.shape)
#   xh = np.empty((sequenceLen+len(h)-1,2))
#   xh[:,0] = np.convolve(x[:,0],h)   
#   xh[:,1] = np.convolve(x[:,1],h)   
  #xh = convolve1d(x,h,axis = 1)
  xh = np.apply_along_axis(lambda m: np.convolve(m,h,mode="full"),axis = 1,arr=x)
  #print(xh.shape)
  SNR = 10**(SNRdB/10)
  sym_noise_pow = np.var(xh)/SNR
  sym_noise_scale_fact = np.sqrt(sym_noise_pow/2)
  if(Noise):
    AWGN = sym_noise_scale_fact*np.random.normal(1, size=xh.shape) 
    xh = xh + AWGN
  #return [xhN.T , np.repeat(y,2) ]
  return [xh ,y ]
  
                        
data={}
def generateData2(m,sequenceLen,trials,Noise,SNRdB):
  [data['x_train'],data['y_train']] = channel2(m=m,sequenceLen=sequenceLen,trials=trials,Noise=Noise,SNRdB=SNRdB)
  [data['x_val'],data['y_val']] = channel2(m=m,sequenceLen=sequenceLen,trials=int(trials*0.5),Noise= Noise,SNRdB=SNRdB)
  [data['x_test'],data['y_test']] = channel2(m=m,sequenceLen=sequenceLen,trials=trials,Noise=Noise,SNRdB=SNRdB)

In [108]:
# Visulizing the data

m = 4         # order of modulation
#seqLen = 64    # length of one trial must be square powers of 2 to make sense see below 
trials = 1000   # number of trials 
Noise = 1
SNRdB = 25 


x = 4
square_power_of_two = 2**(2*x)
seqLen = square_power_of_two  
print(seqLen)

generateData2(m,seqLen,trials,Noise,SNRdB)
print(data['x_train'].shape) # data  # axis 1 should be 2x seqLen because of conv with a 2tap filter
print(data['y_train'].shape) # labels
#print(data['x_train']) # so u can see the data 

# tmp = (data['x_train'][:,:-1].reshape(trials,seqLen))

# #plot 1
# plt.scatter(np.real(tmp[0][:]),np.imag(tmp[0][:]))   
#                                          # I only plot the first 10 points from 
#                                          # the sequence because as time goes on 
#                                          # the data becomes less dependent on the
#                                          # previous symbols. Even 10 is a lot. 
# plt.title('4-QAM Mod With Multi-Channel AWGN plot 1')
# plt.show()

#plot 2
# Just a pure 16 qam with 1 point on each constalation point 
# clearly we have 16 classes
# [xtmp,ytmp]=  qammod(16,data['y_train'])
# plt.scatter(xtmp[:,0],xtmp[:,1])
# plt.title('16-QAM Mod plot 2')
# plt.show()



256
(1000, 262, 2)
(1000, 256)


In [109]:
# Some keras stuffs
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, Flatten, AveragePooling2D, MaxPooling2D, Dropout, BatchNormalization
from keras.layers import Reshape, Permute
from keras import regularizers
import keras
ykeys = {'y_train','y_val','y_test'}
xkeys = {'x_train','x_val','x_test'}

  
y_train = keras.utils.to_categorical(data['y_train'] , num_classes=m)
y_val = keras.utils.to_categorical(data['y_val'] , num_classes=m)
y_test = keras.utils.to_categorical(data['y_test'] , num_classes=m)

print(data['x_train'].shape)
print(data['x_test'].shape)
print(y_train.shape)
print(data['x_val'].shape)


default_data = {
  'x_train': data['x_train'],
  'x_val': data['x_val'],
  'x_test': data['x_test'],
  'y_train': y_train,
  'y_val': y_val,
  'y_test': y_test,
}


(1000, 262, 2)
(1000, 262, 2)
(1000, 256, 4)
(500, 262, 2)


In [118]:
from keras.layers import Reshape,Dense,Embedding,Conv1D, LSTM,CuDNNLSTM, TimeDistributed, GaussianNoise, Softmax,Flatten
from keras import regularizers

modCNN = Sequential()
modCNN.add(Conv1D(40,kernel_size=(40), input_shape=(seqLen+6,2) ))
modCNN.add(Flatten())
modCNN.add(Dense(seqLen*m,kernel_regularizer=regularizers.l2(0.01)))
modCNN.add(Reshape(target_shape = (seqLen,m)))
modCNN.add(Softmax(axis=2))
try_model(modCNN, default_data, 'conv_lstmx', epochs=100)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_29 (Conv1D)           (None, 223, 40)           3240      
_________________________________________________________________
flatten_14 (Flatten)         (None, 8920)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 1024)              9135104   
_________________________________________________________________
reshape_20 (Reshape)         (None, 256, 4)            0         
_________________________________________________________________
softmax_15 (Softmax)         (None, 256, 4)            0         
Total params: 9,138,344
Trainable params: 9,138,344
Non-trainable params: 0
_________________________________________________________________
Train on 1000 samples, validate on 500 samples
Epoch 1/100
1000/1000 [==============================] - 1s 1ms/step - loss: 10.0399 

1000/1000 [==============================] - 0s 379us/step - loss: 1.1854 - acc: 0.7051 - val_loss: 1.2752 - val_acc: 0.6395
Epoch 11/100
1000/1000 [==============================] - 0s 362us/step - loss: 1.1697 - acc: 0.7159 - val_loss: 1.2668 - val_acc: 0.6460
Epoch 12/100
1000/1000 [==============================] - 0s 357us/step - loss: 1.1747 - acc: 0.7168 - val_loss: 1.2703 - val_acc: 0.6477
Epoch 13/100
1000/1000 [==============================] - 0s 366us/step - loss: 1.1664 - acc: 0.7224 - val_loss: 1.2586 - val_acc: 0.6571
Epoch 14/100
1000/1000 [==============================] - 0s 362us/step - loss: 1.1665 - acc: 0.7239 - val_loss: 1.2601 - val_acc: 0.6579
Epoch 15/100
1000/1000 [==============================] - 0s 360us/step - loss: 1.1700 - acc: 0.7271 - val_loss: 1.2566 - val_acc: 0.6659
Epoch 16/100
1000/1000 [==============================] - 0s 366us/step - loss: 1.1402 - acc: 0.7389 - val_loss: 1.2391 - val_acc: 0.6690
Epoch 17/100
1000/1000 [=======================

1000/1000 [==============================] - 0s 368us/step - loss: 1.0803 - acc: 0.7672 - val_loss: 1.2011 - val_acc: 0.6958
Epoch 26/100
1000/1000 [==============================] - 0s 369us/step - loss: 1.0724 - acc: 0.7711 - val_loss: 1.1935 - val_acc: 0.6988
Epoch 27/100
1000/1000 [==============================] - 0s 365us/step - loss: 1.0738 - acc: 0.7709 - val_loss: 1.1929 - val_acc: 0.7011
Epoch 28/100
1000/1000 [==============================] - 0s 360us/step - loss: 1.0709 - acc: 0.7726 - val_loss: 1.1925 - val_acc: 0.7016
Epoch 29/100
1000/1000 [==============================] - 0s 362us/step - loss: 1.0703 - acc: 0.7737 - val_loss: 1.1904 - val_acc: 0.7023
Epoch 30/100
1000/1000 [==============================] - 0s 363us/step - loss: 1.0566 - acc: 0.7805 - val_loss: 1.1889 - val_acc: 0.7019
Epoch 31/100
1000/1000 [==============================] - 0s 376us/step - loss: 1.0487 - acc: 0.7812 - val_loss: 1.1773 - val_acc: 0.7061
Epoch 32/100
1000/1000 [=======================

1000/1000 [==============================] - 0s 365us/step - loss: 0.9768 - acc: 0.8040 - val_loss: 1.1368 - val_acc: 0.7190
Epoch 41/100
1000/1000 [==============================] - 0s 359us/step - loss: 0.9663 - acc: 0.8075 - val_loss: 1.1389 - val_acc: 0.7199
Epoch 42/100
1000/1000 [==============================] - 0s 366us/step - loss: 0.9654 - acc: 0.8082 - val_loss: 1.1321 - val_acc: 0.7214
Epoch 43/100
1000/1000 [==============================] - 0s 361us/step - loss: 0.9566 - acc: 0.8110 - val_loss: 1.1254 - val_acc: 0.7214
Epoch 44/100
1000/1000 [==============================] - 0s 364us/step - loss: 0.9407 - acc: 0.8160 - val_loss: 1.1218 - val_acc: 0.7239
Epoch 45/100
1000/1000 [==============================] - 0s 368us/step - loss: 0.9444 - acc: 0.8131 - val_loss: 1.1188 - val_acc: 0.7246
Epoch 46/100
1000/1000 [==============================] - 0s 366us/step - loss: 0.9363 - acc: 0.8182 - val_loss: 1.1131 - val_acc: 0.7253
Epoch 47/100
1000/1000 [=======================

1000/1000 [==============================] - 0s 366us/step - loss: 0.8608 - acc: 0.8378 - val_loss: 1.0728 - val_acc: 0.7359
Epoch 56/100
1000/1000 [==============================] - 0s 379us/step - loss: 0.8591 - acc: 0.8386 - val_loss: 1.0723 - val_acc: 0.7351
Epoch 57/100
1000/1000 [==============================] - 0s 361us/step - loss: 0.8557 - acc: 0.8398 - val_loss: 1.0767 - val_acc: 0.7347
Epoch 58/100
1000/1000 [==============================] - 0s 358us/step - loss: 0.8405 - acc: 0.8455 - val_loss: 1.0607 - val_acc: 0.7383
Epoch 59/100
1000/1000 [==============================] - 0s 359us/step - loss: 0.8329 - acc: 0.8453 - val_loss: 1.0586 - val_acc: 0.7381
Epoch 60/100
1000/1000 [==============================] - 0s 369us/step - loss: 0.8185 - acc: 0.8519 - val_loss: 1.0498 - val_acc: 0.7399
Epoch 61/100
1000/1000 [==============================] - 0s 374us/step - loss: 0.8129 - acc: 0.8513 - val_loss: 1.0458 - val_acc: 0.7399
Epoch 62/100
1000/1000 [=======================

1000/1000 [==============================] - 0s 366us/step - loss: 0.7677 - acc: 0.8633 - val_loss: 1.0287 - val_acc: 0.7438
Epoch 71/100
1000/1000 [==============================] - 0s 373us/step - loss: 0.7601 - acc: 0.8654 - val_loss: 1.0213 - val_acc: 0.7474
Epoch 72/100
1000/1000 [==============================] - 0s 373us/step - loss: 0.7640 - acc: 0.8653 - val_loss: 1.0256 - val_acc: 0.7448
Epoch 73/100
1000/1000 [==============================] - 0s 366us/step - loss: 0.7620 - acc: 0.8651 - val_loss: 1.0283 - val_acc: 0.7440
Epoch 74/100
1000/1000 [==============================] - 0s 366us/step - loss: 0.7535 - acc: 0.8682 - val_loss: 1.0191 - val_acc: 0.7459
Epoch 75/100
1000/1000 [==============================] - 0s 371us/step - loss: 0.7481 - acc: 0.8692 - val_loss: 1.0188 - val_acc: 0.7465
Epoch 76/100
1000/1000 [==============================] - 0s 369us/step - loss: 0.7370 - acc: 0.8726 - val_loss: 1.0141 - val_acc: 0.7460
Epoch 77/100
1000/1000 [=======================

1000/1000 [==============================] - 0s 374us/step - loss: 0.6948 - acc: 0.8828 - val_loss: 0.9892 - val_acc: 0.7509
Epoch 86/100
1000/1000 [==============================] - 0s 362us/step - loss: 0.6927 - acc: 0.8832 - val_loss: 0.9899 - val_acc: 0.7523
Epoch 87/100
1000/1000 [==============================] - 0s 365us/step - loss: 0.6896 - acc: 0.8848 - val_loss: 0.9860 - val_acc: 0.7514
Epoch 88/100
1000/1000 [==============================] - 0s 377us/step - loss: 0.6839 - acc: 0.8853 - val_loss: 0.9876 - val_acc: 0.7512
Epoch 89/100
1000/1000 [==============================] - 0s 362us/step - loss: 0.6915 - acc: 0.8825 - val_loss: 0.9869 - val_acc: 0.7538
Epoch 90/100
1000/1000 [==============================] - 0s 371us/step - loss: 0.6906 - acc: 0.8846 - val_loss: 0.9939 - val_acc: 0.7520
Epoch 91/100
1000/1000 [==============================] - 0s 369us/step - loss: 0.6856 - acc: 0.8858 - val_loss: 0.9874 - val_acc: 0.7535
Epoch 92/100
1000/1000 [=======================

1000/1000 [==============================] - 0s 159us/step
Test accuracy: [0.9710390005111694, 0.75834765625]
